# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
from ipywidgets.embed import embed_minimal_html


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather = pd.read_csv("../output_data/cities.csv")
weather_df = pd.DataFrame(weather)
weather_df.drop(["Unnamed: 0"], axis = 1, inplace = True)
weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country
0,Avarua,-21.21,-159.78,80.73,77,100,10.47,2020-12-18 05:36:36,CK
1,Hermanus,-34.42,19.23,68.00,72,0,7.00,2020-12-18 05:36:36,ZA
2,Visby,57.64,18.30,44.01,93,40,17.22,2020-12-18 05:36:36,SE
3,Aykhal,66.00,111.50,-31.00,66,0,2.24,2020-12-18 05:36:36,RU
4,Busselton,-33.65,115.33,80.01,32,0,4.00,2020-12-18 05:33:34,AU
...,...,...,...,...,...,...,...,...,...
171,Woodway,31.51,-97.20,44.01,70,0,13.87,2020-12-18 05:36:57,US
172,Romny,50.75,33.47,32.09,98,100,0.87,2020-12-18 05:36:57,UA
173,Inirida,3.87,-67.92,71.76,96,81,1.07,2020-12-18 05:36:57,CO
174,Vostok,46.49,135.88,-3.87,91,100,10.67,2020-12-18 05:36:57,RU


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
fig = gmaps.figure()
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '2px solid black',
    'padding': '2px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
coordinates = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"]
intensity = weather_df["Humidity"].min().astype(float)
heat_map = gmaps.heatmap_layer(coordinates, weights = humidity, dissipating=False, max_intensity= intensity, point_radius=1)
# Get a link for the map figure exported (one version of a screenshot for me)
embed_minimal_html('export.html', views=[fig])
fig.add_layer(heat_map)
fig

Figure(layout=FigureLayout(border='2px solid black', height='600px', margin='0 auto 0 auto', padding='2px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Remove any blank values
weather_df.dropna()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country
0,Avarua,-21.21,-159.78,80.73,77,100,10.47,2020-12-18 05:36:36,CK
1,Hermanus,-34.42,19.23,68.00,72,0,7.00,2020-12-18 05:36:36,ZA
2,Visby,57.64,18.30,44.01,93,40,17.22,2020-12-18 05:36:36,SE
3,Aykhal,66.00,111.50,-31.00,66,0,2.24,2020-12-18 05:36:36,RU
4,Busselton,-33.65,115.33,80.01,32,0,4.00,2020-12-18 05:33:34,AU
...,...,...,...,...,...,...,...,...,...
171,Woodway,31.51,-97.20,44.01,70,0,13.87,2020-12-18 05:36:57,US
172,Romny,50.75,33.47,32.09,98,100,0.87,2020-12-18 05:36:57,UA
173,Inirida,3.87,-67.92,71.76,96,81,1.07,2020-12-18 05:36:57,CO
174,Vostok,46.49,135.88,-3.87,91,100,10.67,2020-12-18 05:36:57,RU


In [5]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.

ideal_df = weather_df.loc[(weather_df["Max Temp"] >= 70) & (weather_df["Max Temp"] < 80) & (weather_df["Wind Speed"] < 10) & (weather_df["Cloudiness"] == 0)]
# Drop any duplicates
ideal_df.drop_duplicates()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country
57,Maceio,-9.67,-35.74,73.4,100,0,2.24,2020-12-18 05:36:41,BR
156,Neuquen,-38.95,-68.06,73.4,33,0,8.05,2020-12-18 05:36:55,AR


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_df.copy()
# Set values for new column as blank
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country,Hotel Name
57,Maceio,-9.67,-35.74,73.4,100,0,2.24,2020-12-18 05:36:41,BR,
156,Neuquen,-38.95,-68.06,73.4,33,0,8.05,2020-12-18 05:36:55,AR,


In [7]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"keyword": "hotel",
          "type": "hotel",
          "radius" : 5000,
          "key" : g_key} 

# Gather lat and lng for each city to add to params and search for hotels; add Hotel Name if found
for index, row in hotel_df.iterrows():
    city = row["City"]
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat}, {lng}" 

    response = requests.get(url, params = params).json()
    nearby_hotels = response["results"]
    try:
        print(f"Trying {city} now...")
        hotel_df.loc[index, "Hotel Name"] = nearby_hotels[0]["name"]
    except (KeyError, IndexError):
        print("Skipping")
    time.sleep(1.1)
    
hotel_df

Trying Maceio now...
Skipping
Trying Neuquen now...
Skipping


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country,Hotel Name
57,Maceio,-9.67,-35.74,73.4,100,0,2.24,2020-12-18 05:36:41,BR,
156,Neuquen,-38.95,-68.06,73.4,33,0,8.05,2020-12-18 05:36:55,AR,


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))